## Creating sub-agent 3: Market analysis agent

In this folder we will create the second sub-agent for get options chains, insider trading transactions and get news. We will use the `financial dataset API` and three functions: `get_options_chains`, `get_insider_trading` and `get_news` using the `marketing-analyst-agent`.

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Import required libraries

Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:

1. `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties.

1. `invoke_agent_helper`: helps you to invoke your agent using invoke_agent
You can see the implementation of both functions in the parent directory

In [ ]:
import os
import sys
import time
import json
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from agents import create_agent, invoke_agent_helper

In [ ]:
# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [ ]:
agent_instruction = """You are a comprehensive market analysis assistant that helps users analyze options chains, insider trading data, and relevant market news.
You require the user to provide a stock ticker symbol for analysis.
If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.

You can perform the following types of analysis. You have access to APIs and tools to call to fetch data based on what the user question is:

1. Use the appropriate functions based on the analysis needed:
    - get_options_chain for options analysis
    - get_insider_trades for insider trading analysis
    - get_news for market news and sentiment

2. Options Chain Analysis:
   - View available options contracts
   - Filter by strike price
   - Filter by option type (call/put)
   - Analyze options pricing and volume

3. Insider Trading Analysis:
   - Recent insider transactions
   - Transaction types (buy/sell)

4. Market News Analysis:
   - Latest relevant news
   - Market sentiment
   - Industry trends

For options chain data, you can specify:
- Strike price filters
- Option type (call/put)
- Number of results to return (limit)

For insider trades, you can specify:
- Number of transactions to analyze (limit)

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above.
"""

agent_description = "Agent for analyzing options chains, insider trading data, and market news"

In [ ]:
# These are the functions that are used by the market analysis agent
functions = [{
    'name': 'get_options_chain',
    'description': 'Get options chain data for a ticker',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of options to retrieve",
            "required": False,
            "type": "integer"
        },
        "strike_price": {
            "description": "filter by strike price",
            "required": False,
            "type": "number"
        },
        "option_type": {
            "description": "filter by option type (call/put)",
            "required": False,
            "type": "string"
        }
    }
},
{
    'name': 'get_insider_trades',
    'description': 'Get insider trading transactions for a ticker',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of transactions to retrieve",
            "required": False,
            "type": "integer"
        }
    }
},
{
    'name': 'get_news',
    'description': 'Get latest market news and analysis',
    'parameters': {
        "query": {
            "description": "search query for news",
            "required": True,
            "type": "string"
        },
        "max_results": {
            "description": "maximum number of news results",
            "required": False,
            "type": "integer"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests` to make requests to the financial data API key.

In [ ]:
# Create and publish the layer 
layer_zip = create_lambda_layer(['requests', 'tavily-python'])
layer_arn = publish_layer('marketing-agent-lambda-layer')

### Create agent
---

Next we will create the agent with the provided information

In [ ]:
action_group_config = {
    'name': 'MarketingAnalysisActionGroup',
    'description': 'Action group to analyze marketing questions from the user',
    'functions': functions,
    'lambda_function_name': f'{SUB_AGENT_MARKETING_ANALYST}-lambda',
    'lambda_file_path': MARKET_ANALYSIS_LAMBDA_FUNCTION_NAME,
    'dynamodb_table_name': f'{SUB_AGENT_MARKETING_ANALYST}-table',
    'dynamodb_attribute_name': 'analysis-id', 
    # Adding the lambda layer that contains the requests installation
    'lambda_layers': [layer_arn]
}

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = action_group_config['lambda_function_name']
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY'), 
    'TAVILY_API_KEY': os.getenv('TAVILY_API_KEY')
}

# Create the agent with the updated action group config
marketing_agent_id, marketing_agent_alias_id, marketing_agent_alias_arn = create_agent(
    SUB_AGENT_MARKETING_ANALYST,
    agent_instruction,
    agent_foundation_model=BEDROCK_MODEL_CLAUDE_HAIKU,
    agent_description=agent_description,
    action_group_config=action_group_config
)

In [ ]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset and Tavily API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=action_group_config['lambda_layers']
)

### Getting details from the agent
--- 
Let's take a look at the details from the created agent.

In [ ]:
marketing_agent_id, marketing_agent_alias_id, marketing_agent_alias_arn

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our invoke_agent_helper function. Here, we will invoke the `fundamental analyst` agent to provide balance sheet information, income statements and cash flow information based on the question provided by the user.

In [ ]:
# Sleep for 30 seconds before invoking the marketing analyst agent
time.sleep(30)

### Route the agent to call the `get_news()` function
---

This function enables the agent with a `TAVILY_API_KEY` that enables web search. In our function, we have provided `google.com` and `bloomberg.com` as domains that the agent can access, so the agent will search for results in these domains and return a comprehensive answer. 

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Can you give any news on 'AMZN' and what is its stock like? Search for 'Amazon stock'. Be concise and only search once."""
response = invoke_agent_helper(
    query, session_id, marketing_agent_id, marketing_agent_alias_id, enable_trace=True
)
print(response)

### Route the agent to call the `get_insider_trades()` function
---

In this question, we ask the agent about `How many shares of Apple has General Counsel sold over the past year?`. With this question, the agent identifies that `get_insider_trades()` is the right function to call and using the `FINANCIAL_DATASET_API` key, retrieves that information to answer the user question.

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """How many shares of AAPL has General Counsel sold over the past year?"""
response = invoke_agent_helper(
    query, session_id, marketing_agent_id, marketing_agent_alias_id, enable_trace=True
)
print(response)

### Saving the information
---
Let's now save some information so that we can use this agent as a sub-agent of our financial hedge fund assistant

In [ ]:
%store marketing_agent_id
%store marketing_agent_alias_id
%store marketing_agent_alias_arn